In [147]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import  train_test_split
from textblob import TextBlob
import numpy as np
from lib.ops import *
import tensorflow as tf

ModuleNotFoundError: No module named 'lib'

In [6]:
# open twitter sentiment data
# 1 = positive
# 0 = Negative
twitter_data_df = pd.read_csv("../data/raw/Twitter_Sentiment_Analysis/Sentiment Analysis Dataset.csv",error_bad_lines=False)

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [28]:
blob.tokenize()

WordList(['..', 'Omgaga', '.', 'Im', 'sooo', 'im', 'gunna', 'CRy', '.', 'I', "'ve", 'been', 'at', 'this', 'dentist', 'since', '11..', 'I', 'was', 'suposed', '2', 'just', 'get', 'a', 'crown', 'put', 'on', '(', '30mins', ')', '...'])

# Load Data

In [7]:
twitter_data_df.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [11]:
X_all = twitter_data_df.SentimentText

In [12]:
y_all = twitter_data_df.Sentiment

In [9]:
twitter_data_df.shape

(1578612, 4)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, test_size=.25)

In [15]:
X_train.shape

(1183959,)

In [16]:
y_train.shape

(1183959,)

### Once this is done, we need to convert the sentences as a one-hot tensor of shape [sentence_length x word_length x alphabet_size].
- https://charlesashby.github.io/2017/06/05/sentiment-analysis-with-char-lstm/

In [41]:
EMB_ALPHABET = 'abcdefghijklmnopqrstuvwxyz0123456789-,;' \
               '.!?:\'"/\\|_@#$%^&*~`+-=<>()[]{}'
MAX_WORD_LENGTH = 16 # number of characters in a word
ALPHABET_SIZE = len(emb_alphabet)
# we associate every character in our alphabet to a number: 
# e.g. b => 1 d => 3 etc.
ALPHABET_DICT = {ch: ix for ix, ch in enumerate(emb_alphabet)}

In [110]:
def encode_one_hot( sentence, emb_alphabet, max_word_length, alphabet_size, alphabet_dict):
    """Convert a sentence to a one hot character encoding tensor for that sentence using alphanumeric characters"""
    # https://charlesashby.github.io/2017/06/05/sentiment-analysis-with-char-lstm/
    # Convert Sentences to np.array of Shape 
    # ('sent_length', 'word_length', 'emb_size')

    sent = []

    # We need to keep track of the maximum length of the sentence in a minibatch
    # so that we can pad them with zeros, this is why we return the length of every
    # sentences after they are converted to one-hot tensors
    SENT_LENGTH = 0

    # Here, we remove any non-printable characters in a sentence (mostly
    # non-ASCII characters)
    printable = emb_alphabet
    encoded_sentence = filter(lambda x: x in printable, sentence)

    # word_tokenize() splits a sentence into an array where each element is
    # a word in the sentence, for example, 
    # "My name is Charles" => ["My", "name", "is", Charles"]
    blob = TextBlob(sentence)
    individual_words_from_sentence = blob.tokenize()
    for word in individual_words_from_sentence :

        # Encode one word as a matrix of shape [max_word_length x ALPHABET_SIZE]
        word_encoding = np.zeros(shape=(max_word_length, alphabet_size))

        for i, char in enumerate(word):

            # If the character is not in the alphabet, ignore it    
            try:
                char_encoding = alphabet_dict[char]
                one_hot = np.zeros(ALPHABET_SIZE)
                one_hot[char_encoding] = 1
                word_encoding[i] = one_hot

            except Exception as e:
                pass

        sent.append(np.array(word_encoding))
        SENT_LENGTH += 1

    return np.array(sent), SENT_LENGTH

In [111]:
array,length = encode_one_hot(X_all[1], EMB_ALPHABET , MAX_WORD_LENGTH, ALPHABET_SIZE, ALPHABET_DICT)
# array is number of words in sentence X Max word length  X one hot encoding for alphabet

# Pad tensors to all have the same size 
- [batch_size x maximum_sentence_length x maximum_word_length x alphabet_size]

In [68]:
y_all[0]

0

In [72]:
X_all[0][2:-1]

'                   is so sad for my APL friend............'

In [155]:
def make_minibatch( sentences, sentiment_y,  max_word_length, alphabet_size):
    """Create a minibath of one-hot encoded sentences 
    array is Batch Size X max number of words in sentence X Max word length  X one hot encoding for alphabet,
    
    and one hot encoded y [0,1] or [1,0] whre 1=positive"""
    # Create a minibatch of sentences and convert sentiment
    # to a one-hot vector, also takes care of padding

    max_word_length = max_word_length
    minibatch_x = []
    minibatch_y = []
    max_length = 0

    for sentence,sent in zip(sentences, sentiment_y):
        # Append the one-hot encoding of the sentiment to the minibatch of Y
        # 0: Negative 1: Positive
        minibatch_y.append(np.array([0, 1]) if sent == 0 else np.array([1, 0]))

        # One-hot encoding of the sentence
        one_hot, length = encode_one_hot(sentence,EMB_ALPHABET, MAX_WORD_LENGTH, ALPHABET_SIZE, ALPHABET_DICT )

        # Calculate maximum_sentence_length
        if length >= max_length:
            max_length = length

        # Append encoded sentence to the minibatch of X
        minibatch_x.append(one_hot)


    # data is a np.array of shape ('b', 's', 'w', 'e') we want to
    # pad it with np.zeros of shape ('e',) to get 
    # ('b', 'SENTENCE_MAX_LENGTH', 'WORD_MAX_LENGTH', 'e')
    def numpy_fillna(data):
        """ This is a very useful function that fill the holes in our tensor """

        # Get lengths of each row of data
        lens = np.array([len(i) for i in data])

        # Mask of valid places in each row
        mask = np.arange(lens.max()) < lens[:, None]
        #print(mask)

        # Setup output array and put elements from data into masked positions
        out = np.zeros(shape=(mask.shape + (max_word_length, alphabet_size)),
                       dtype='float32')
        #print(out)

        out[mask] = np.concatenate(data)
        #print(out,'final')
        return out

    # Padding...
    minibatch_x = numpy_fillna(minibatch_x)

    return minibatch_x, np.array(minibatch_y)

In [157]:
minibatch_x, minibatch_y = make_minibatch(X_all[:15],y_all[:15], MAX_WORD_LENGTH, ALPHABET_SIZE)

In [158]:
minibatch_x.shape

(15, 31, 16, 69)

## Make a generator to feed to our network

In [150]:
def iterate_minibatch(batch_size, X_data, y_data):
        """ Returns Next Batch """
        
        n_samples = len(X_data)
        
        # Number of batches / number of iterations per epoch
        n_batch = int(n_samples // batch_size)
        
        # Creates a minibatch, loads it to RAM and feed it to the network
        # until the buffer is empty
        for i in range(n_batch):

            inputs, targets = make_minibatch(X_data[max(batch_size*i-batch_size,0):max(batch_size*i,batch_size)],\
                                             y_data[max(batch_size*i-batch_size,0):max(batch_size*i,batch_size)],\
                                             MAX_WORD_LENGTH, ALPHABET_SIZE)
            yield inputs, targets

In [151]:
next(iterate_minibatch(16,X_train,y_train))

(array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         ..., 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,

# Define the network

In [144]:


def conv2d(input_, output_dim, k_h, k_w, name="conv2d"):
    """ Straight-forward convvolutional layer """
    # w is the kernel, b the bias, no strides and VALID padding

    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim])
        b = tf.get_variable('b', [output_dim])

    return tf.nn.conv2d(input_, w, strides=[1, 1, 1, 1], padding='VALID') + b

In [148]:
def tdnn(input_, kernels, kernel_features, scope='TDNN'):
    ''' Time Delay Neural Network
    :input:           input float tensor of shape 
                      [(batch_size*num_unroll_steps) x max_word_length x embed_size]
    :kernels:         array of kernel sizes
    :kernel_features: array of kernel feature sizes (parallel to kernels)
    '''
    assert len(kernels) == len(kernel_features), 'Kernel and Features must have the same size'

    # input_ is a np.array of shape ('b', 'sentence_length', 'max_word_length', 'embed_size') we
    # need to convert it to shape ('b * sentence_length', 1, 'max_word_length', 'embed_size') to
    # use conv2D
    # It might not seem obvious why we need to use this small hack at first sight, the reason
    # is that sentence_length will change across the different minibatches, but if we kept it
    # as is sentence_length would act as the number of channels in the convnet which NEEDS to
    # stay the same
    input_ = tf.reshape(input_, [-1, self.max_word_length, ALPHABET_SIZE])
    input_ = tf.expand_dims(input_, 1)

    layers = []
    with tf.variable_scope(scope):
        for kernel_size, kernel_feature_size in zip(kernels, kernel_features):
            reduced_length = self.max_word_length - kernel_size + 1

            # [batch_size * sentence_length x max_word_length x embed_size x kernel_feature_size]
            conv = conv2d(input_, kernel_feature_size, 1,
                          kernel_size, name="kernel_%d" % kernel_size)

            # [batch_size * sentence_length x 1 x 1 x kernel_feature_size]
            pool = tf.nn.max_pool(tf.tanh(conv), [1, 1, reduced_length, 1], [1, 1, 1, 1], 'VALID')

            layers.append(tf.squeeze(pool, [1, 2]))

        if len(kernels) > 1:
            output = tf.concat(layers, 1)
        else:
            output = layers[0]

    return output

In [118]:
minibatch_x.shape

(25, 39, 16, 69)

In [93]:
minibatch_x.shape

(10, 31, 16, 69)

In [86]:
x_1 , l = encode_one_hot(X_all[1],EMB_ALPHABET, MAX_WORD_LENGTH, ALPHABET_SIZE, ALPHABET_DICT )

In [87]:
x_2 , l = encode_one_hot(X_all[2],EMB_ALPHABET, MAX_WORD_LENGTH, ALPHABET_SIZE, ALPHABET_DICT )

In [88]:
x_1.shape

(7, 16, 69)

In [89]:
x_2.shape

(6, 16, 69)

# Goal is to train two models - LSTM and CNN to sentiment analysis
- Using tweets here because the primary source of text from founders will be Tweets